In [3]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [16]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [17]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
print('The dimensions of the original dataframe are: ', df.shape)

The dimensions of the original dataframe are:  (288, 3)


### Only cells with an assigned borough are processed. Cells where the boroughs are not assigned are neglected.

In [19]:
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=['Borough'], inplace = True)

In [20]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [21]:
print('The new dimensions of the dataframe are: ', df.shape)

The new dimensions of the dataframe are:  (211, 3)


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [23]:
df=df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [24]:
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [25]:
df.shape

(103, 3)

In [26]:
print('The final dimensions of the processed data frame are: ', df.shape)

The final dimensions of the processed data frame are:  (103, 3)


## Obtaining the lat-long coordinates of each neighborhood

In [27]:
df1=pd.read_csv('https://cocl.us/Geospatial_data')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df1=df1.rename(columns={"Postal Code": "Postcode"})
df2=pd.merge(df, df1, how ='inner', on ='Postcode')
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [31]:
from geopy.geocoders import Nominatim
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [32]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'],df2['Longitude'],df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [33]:
CLIENT_ID = 'U22Z53NJJ1SUU2XXIRYVRKSBFO3E3RA20TU1RZKG5DISHSDQ' # Foursquare ID
CLIENT_SECRET = 'PFMWLP2202SILPSEWF3XJ2V4JYDIPV3GT2OTNLHT02OUCUN1' # your Foursquare Secret
VERSION = '20180801' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U22Z53NJJ1SUU2XXIRYVRKSBFO3E3RA20TU1RZKG5DISHSDQ
CLIENT_SECRET:PFMWLP2202SILPSEWF3XJ2V4JYDIPV3GT2OTNLHT02OUCUN1


In [35]:
downtown_to = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_to.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [37]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of are 43.654027, -79.3802003.


In [39]:
downtown_to.loc[0, 'Neighbourhood']

'Rosedale'

In [40]:
neigh_latitude = downtown_to.loc[0, 'Latitude'] # neighborhood latitude value
neigh_longitude = downtown_to.loc[0, 'Longitude'] # neighborhood longitude value
neigh_name = downtown_to.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_latitude, 
                                                               neigh_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [46]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 750 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_latitude, 
    neigh_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=U22Z53NJJ1SUU2XXIRYVRKSBFO3E3RA20TU1RZKG5DISHSDQ&client_secret=PFMWLP2202SILPSEWF3XJ2V4JYDIPV3GT2OTNLHT02OUCUN1&v=20180801&ll=43.6795626,-79.37752940000001&radius=750&limit=100'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dcafc829da7ee001c9cdf59'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.68631260675, 'lng': -79.36821347411556},
   'sw': {'lat': 43.67281259324999, 'lng': -79.38684532588447}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0960fb009a0f472fabe7bf',
       'name': 'Craigleigh Gardens',
       'location': {'address': '160 South Drive',
        'crossStreet': 'at Elm Ave',
        'lat': 43.67809940868806,
        'lng': -79.37158584594727,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67809940868806,
          'lng': -79.37158584594727}],
        'distance': 505

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Craigleigh Gardens,Park,43.678099,-79.371586
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
venues = getNearbyVenues(names=downtown_to['Neighbourhood'],
                                   latitudes=downtown_to['Latitude'],
                                   longitudes=downtown_to['Longitude']
                                  )

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [56]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",25,25,25,25,25,25
"Cabbagetown,St. James Town",73,73,73,73,73,73
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,30,30,30,30,30,30
Church and Wellesley,100,100,100,100,100,100
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [57]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Tram Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Tram Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.000000,...,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00
1,Berczy Park,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.00,0.04,0.04,0.08,0.12,0.04,0.00,0.000000,...,0.000000,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,"Cabbagetown,St. James Town",0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Central Bay Street,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,...,0.000000,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.00
5,"Chinatown,Grange Park,Kensington Market",0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.000000,0.00,0.01,0.00,0.03,0.00,0.00,0.04,0.01,0.00
6,Christie,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Church and Wellesley,0.010000,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,...,0.000000,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.00,0.01
8,"Commerce Court,Victoria Hotel",0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.000000,...,0.000000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.000000,...,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00


In [59]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.06
2        Steakhouse  0.04
3  Sushi Restaurant  0.04
4           Theater  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.06
2        Hotel  0.05
3         Café  0.05
4     Beer Bar  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.12
1   Harbor / Marina  0.12
2     Boat or Ferry  0.08
3  Sculpture Garden  0.08
4       Coffee Shop  0.08


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.08
1  Pizza Place  0.05
2         Park  0.05
3   Restaurant  0.05
4         Café  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.07
1       Ice Cream Shop  0.04
2   Italian Restaurant  0.04
3          Art Gallery  0.03
4  Japanese Restaurant  0.03


----Chinatown,Grange Park,Kens

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Sushi Restaurant,Asian Restaurant,Gastropub,Pizza Place,Restaurant,Hotel,Bar
1,Berczy Park,Coffee Shop,Restaurant,Hotel,Café,Beer Bar,Italian Restaurant,Park,Gastropub,Cocktail Bar,Japanese Restaurant
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Harbor / Marina,Coffee Shop,Sculpture Garden,Rental Car Location,Boat or Ferry,Airport Lounge,Boutique,Plane,Bar
3,"Cabbagetown,St. James Town",Coffee Shop,Pizza Place,Restaurant,Park,Café,Grocery Store,Pharmacy,Pub,Convenience Store,Italian Restaurant
4,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Art Gallery,Café,Cosmetics Shop,Bakery,Bar,Clothing Store


In [62]:
# set number of clusters
kclusters = 6

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 4, 0, 4, 5, 0, 2, 2], dtype=int32)

## Add cluster labels

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged =  downtown_to

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Trail,Playground,Candy Store,Wings Joint,Diner,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,4,Coffee Shop,Pizza Place,Restaurant,Park,Café,Grocery Store,Pharmacy,Pub,Convenience Store,Italian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Diner,Italian Restaurant,Men's Store,Mediterranean Restaurant,Grocery Store,Gay Bar,Gastropub
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Italian Restaurant,Park,Restaurant,Bakery,Café,Pub,Theater,Mexican Restaurant,Performing Arts Venue
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Gastropub,Ramen Restaurant,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Department Store,Sporting Goods Shop
